# Topic Modeling: Non-Matrix Factorisation (NMF)

In [1]:
# from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd
from tqdm import tqdm
# from gpt4all import GPT4All
from openai import OpenAI
import json
import re

# Load Data

In [2]:
df = pd.read_parquet('database/aspect_topic_db.parquet') 
print(df.shape)
display(df.head())

(16951, 7)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment   aspect_topic  
0   neutral  inquire__card  
1   neutral  inquire__card  
2   neutral  inquire__card  
3  positive  inquire__card  
4  positive  inquire__card

In [3]:
# Convert to list
ds_sgd_text = df['text'].tolist()
for i in range(5):
    print(ds_sgd_text[i])

i am still waiting on my card
what can i do if my card still hasnt arrived after 2 weeks
i have been waiting over a week is the card still coming
can i track my card while it is in the process of delivery
how do i know if i will get my card or if it is lost


# Vectorise Dialog Text

In [4]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(
    stop_words='english',  # Remove common English stopwords
    max_features=5000,     # Consider using a larger number of features for bigger datasets
    ngram_range=(1, 2),    # Use unigrams and bigrams
    max_df=0.95,           # Ignore terms that appear in more than 95% of the documents
    min_df=2               # Ignore terms that appear in less than 2 documents
)
X_tfidf = tfidf_vectorizer.fit_transform(ds_sgd_text)

# Use NMF for Topic Modeling

### Define Number of Topics

In [5]:
# Define the number of topics
n_topics = 20

In [6]:
nmf_model = NMF(n_components=n_topics, random_state=42)  # components means topics
W = nmf_model.fit_transform(X_tfidf)  # Document-topic matrix aka Features
H = nmf_model.components_             # Topic-term matrix aka Components aka Topics
print(W.shape, H.shape)

(16951, 20) (20, 5000)


In [7]:
# num_documents_per_topic = np.sum(W > 0, axis=0)
# print(num_documents_per_topic.shape)
# for topic_num in range(len(num_documents_per_topic)):
#     print(f"topic_num: {topic_num}\tnum of docs: {num_documents_per_topic[topic_num]} ")

# Show Topics

In [8]:
def print_topics(H, W, feature_names, n_top_words=10):
    num_documents_per_topic = np.sum(W > 0, axis=0)
    
    for topic_idx, topic in enumerate(H):
        print(f"Topic #{topic_idx}:")
        print(f"Num Documents in Topic: {num_documents_per_topic[topic_idx]}")
        print("top features in topic:")
        top_features = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print(", ".join(top_features))
        print()

# Print the topics with top words
feature_names = tfidf_vectorizer.get_feature_names_out()
print_topics(H, W, feature_names, n_top_words=10)

Topic #0:
Num Documents in Topic: 8061
top features in topic:
account, transfer, transfer account, delete, delete account, need, open, open account, bank, bank transfer

Topic #1:
Num Documents in Topic: 8416
top features in topic:
card, credit, credit card, activate card, using card, using, physical, physical card, pin, activate

Topic #2:
Num Documents in Topic: 4586
top features in topic:
assist today, assist, today, thank calling, calling, thank, bank, alex assist, calling bank, hello thank

Topic #3:
Num Documents in Topic: 5918
top features in topic:
exchange, rate, exchange rate, wrong, exchange rates, currency, rates, rate wrong, foreign, currencies

Topic #4:
Num Documents in Topic: 5494
top features in topic:
checking account, checking, account, balance checking, check checking, send, use checking, want, check balance, dollars

Topic #5:
Num Documents in Topic: 5648
top features in topic:
virtual, virtual card, disposable, disposable virtual, card, work, card work, order, ord

In [9]:
def create_topics_df(H, W, feature_names, n_top_words=10):
    topic_lst = []
    num_of_docs_per_topic = np.sum(W > 0, axis=0)  # num of docs per topic
    term_importance = np.sum(H, axis=1)  # significance of terms for each topic
    topic_importance = num_of_docs_per_topic + term_importance  # to decide later if should use weights for the terms
    
    top_n_words = []

    for topic_idx, topic in enumerate(H):
        topic_lst.append(topic_idx)        
        top_features = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        top_words = ", ".join(top_features)
        top_n_words.append(top_words)
    data_0 = {
        "topic_num": topic_lst, 
        "topic_importance": topic_importance, 
        "num_of_docs_per_topic": num_of_docs_per_topic,
        "term_importance" : term_importance,
        "features": top_n_words}
    return pd.DataFrame(data_0).sort_values(by=['topic_importance'], ascending=False)

df_nmf = create_topics_df(H, W, feature_names, n_top_words=10)
df_nmf.reset_index(drop=True, inplace=True)
print(df_nmf.shape)
display(df_nmf.head(10))

(20, 5)


topic_num  topic_importance  num_of_docs_per_topic  term_importance  \
0         11       8778.383099                   8738        40.383099   
1          1       8438.417581                   8416        22.417581   
2         18       8430.757118                   8386        44.757118   
3          9       8406.854512                   8344        62.854512   
4          0       8134.452789                   8061        73.452789   
5         14       7692.998616                   7640        52.998616   
6         13       7651.844081                   7623        28.844081   
7         17       6350.935129                   6332        18.935129   
8          6       6247.718334                   6234        13.718334   
9          3       5937.257806                   5918        19.257806   

                                            features  
0  number, sure, account number, 123456789, numbe...  
1  card, credit, credit card, activate card, usin...  
2  help, thank, thanks, ll, appreciate, alex, tha...  
3  frustrated, really, really frustrated, right, ...  
4  account, transfer, transfer account, delete, d...  
5  hi, great, bank, ve, hope, day, heard, great d...  
6  app, mobile, mobile app, card app, using, phon...  
7  deposit, fixed, fixed deposit, cash deposit, c...  
8  savings, savings account, account, balance sav...  
9  exchange, rate, exchange rate, wrong, exchange...

# Generate Most Likely Topic, Trigger
using LLM - OpenAI

In [10]:
from pathlib import Path
from dotenv import load_dotenv
import os

cwd = Path.cwd()
env_path = cwd.parent.parent.parent.parent
print(env_path)

load_dotenv(dotenv_path=env_path / '.env')
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key: Found")
else:
    print("API Key not found. Please ensure it is in the .env file.")

client = OpenAI()

D:\OneDrive\RChai\Documents\NUS_ISS_Prjs\4_PLP
OpenAI API Key: Found


In [11]:
def make_prompt(feature_list: str):
    p1 = """Given a list of words related to the Banking sector, give me the most likely topic,  """
    p2 = """a short concise statement on what might have triggered the topic (the trigger), """
    p3 = """and a short explanation. Please reply using in JSON format using the following keys: """
    p4 = """'topic', 'trigger', 'explanation'. Do not reply with anything else. """
    p5 = """Give stronger emphasis to any verbs in the word list. If there are more than one verb in the list, """
    p6 = """give stronger emphasis to earlier verbs.  The word list is: """
    return p1 + p2 + p3 + p4 + p5 + p6 + "\n"+ feature_list 

make_prompt("apple, orange, banana")

"Given a list of words related to the Banking sector, give me the most likely topic,  a short concise statement on what might have triggered the topic (the trigger), and a short explanation. Please reply using in JSON format using the following keys: 'topic', 'trigger', 'explanation'. Do not reply with anything else. Give stronger emphasis to any verbs in the word list. If there are more than one verb in the list, give stronger emphasis to earlier verbs.  The word list is: \napple, orange, banana"

In [12]:
df_nmf_topN = df_nmf.iloc[:n_topics, :].copy()
df_nmf_topN.shape, df_nmf_topN.columns 

((20, 5),
 Index(['topic_num', 'topic_importance', 'num_of_docs_per_topic',
        'term_importance', 'features'],
       dtype='object'))

In [13]:
def get_dict_from_openai_completion(c):
    c_content = c.choices[0].message.content
    match = re.search(r'\{.*?\}', c_content, re.DOTALL)
    if match:
        json_str = match.group(0)
        return json.loads(json_str)
    else:
        print("No match found")
        print(c)
        print('*' * 50)

In [14]:
completion_lst = []
import time

for i in tqdm(range(n_topics)):
    prompt = make_prompt(df_nmf_topN.loc[i, 'features'])
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    completion_lst.append(get_dict_from_openai_completion(completion))
    time.sleep(0.5)
    
print('\n\nall tasks completed\n\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:59<00:00,  2.98s/it]



all tasks completed




In [15]:
topic_lst = [i['topic'] for i in completion_lst]
trigger_lst = [i['trigger'] for i in completion_lst]
explain_lst = [i['explanation'] for i in completion_lst]

df_nmf_topN['topic'] = topic_lst
df_nmf_topN['trigger'] = trigger_lst
df_nmf_topN['explanation'] = explain_lst

In [16]:
df_nmf_topN.head(5)

topic_num  topic_importance  num_of_docs_per_topic  term_importance  \
0         11       8778.383099                   8738        40.383099   
1          1       8438.417581                   8416        22.417581   
2         18       8430.757118                   8386        44.757118   
3          9       8406.854512                   8344        62.854512   
4          0       8134.452789                   8061        73.452789   

                                            features  \
0  number, sure, account number, 123456789, numbe...   
1  card, credit, credit card, activate card, usin...   
2  help, thank, thanks, ll, appreciate, alex, tha...   
3  frustrated, really, really frustrated, right, ...   
4  account, transfer, transfer account, delete, d...   

                                           topic  \
0                      Ensuring Account Security   
1                         Credit Card Activation   
2                  Customer Support Appreciation   
3  Customer Service Challenges in Online Banking   
4                      Account Management Issues   

                                             trigger  \
0                                               make   
1              Customers activating new credit cards   
2                                      Thanking Help   
3  Customers have been trying to perform transact...   
4                                     Delete Account   

                                         explanation  
0  The strong emphasis on the verbs 'make' and 'm...  
1  The focus on 'activate card' and related verbs...  
2  The words 'thank', 'thanks', and 'appreciate' ...  
3  The word 'trying' indicates that customers are...  
4  The list of words suggests issues pertaining t...

### Save to Disk before Sentiment Classification

In [17]:
df_nmf_topN.to_csv(f"output_data/topic_modeling_nmf_top_{n_topics}.csv", index=False)
df_nmf_topN.to_parquet(f"output_data/topic_modeling_nmf_top_{n_topics}.parquet", index=False)

# Sentiment Classification

In [18]:
import pandas as pd
from pathlib import Path
import sys

common_code_path = Path.cwd().parent.parent.parent / "plp_common_code"

if str(common_code_path) not in sys.path:
    sys.path.append(str(common_code_path))
from amhere import foundme
foundme()  # this fn will fail if common code path is not found or !exists
del foundme
del common_code_path

from clf_Helper import load_hf_pdt_clf

In [19]:
df_nmf_topN.head(3)

topic_num  topic_importance  num_of_docs_per_topic  term_importance  \
0         11       8778.383099                   8738        40.383099   
1          1       8438.417581                   8416        22.417581   
2         18       8430.757118                   8386        44.757118   

                                            features  \
0  number, sure, account number, 123456789, numbe...   
1  card, credit, credit card, activate card, usin...   
2  help, thank, thanks, ll, appreciate, alex, tha...   

                           topic                                trigger  \
0      Ensuring Account Security                                   make   
1         Credit Card Activation  Customers activating new credit cards   
2  Customer Support Appreciation                          Thanking Help   

                                         explanation  
0  The strong emphasis on the verbs 'make' and 'm...  
1  The focus on 'activate card' and related verbs...  
2  The words 'thank', 'thanks', and 'appreciate' ...

##### Classify sentiment based on `trigger`

In [24]:
sentiment_clf = load_hf_pdt_clf(
            nlp_task='text-classification',
            hf_model_chkpt="richardchai/plp_sentiment_clr_distilbert")

In [25]:
def predict_sentiment(x: [str, list]):
    response = sentiment_clf(x)
    sentiment = [r['label'] for r in response]

    if isinstance(sentiment, list) and len(sentiment) < 2:
        return sentiment[0]
    else:
        return str(sentiment)[1:-1]
df_nmf_topN['sentiment'] = df_nmf_topN['trigger'].apply(lambda x: predict_sentiment(x))
df_nmf_topN.loc[:, ['trigger', 'sentiment']]

trigger      sentiment
0                                                make       positive
1               Customers activating new credit cards       positive
2                                       Thanking Help       positive
3   Customers have been trying to perform transact...        neutral
4                                      Delete Account       positive
5   Bank reached out to customers with positive en...  very positive
6   Increased use of mobile applications for finan...       positive
7       Increasing Interest in Fixed Deposit Accounts       positive
8                                                send       positive
9                                          rate wrong        neutral
10       Customers need to activate new credit cards.       positive
11  Increased demand for secure and flexible payme...  very negative
12  Multiple customers reported that they didn't g...        neutral
13  Customers send requests to check their account...       positive
14  Increased customer inquiries about account bal...       positive
15                                       card payment       positive
16  Recent updates or issues in online banking ser...       positive
17  Rising use of banking apps and cards for trans...        neutral
18                       The need to assist customers       positive
19                                    didnt recognize        neutral

# Save to Disk After Sentiment Classification & Overwrite (this is a calculation, saving is just caching for dashboard to display)
### so save to data_output (note: in this testing notebook, we save to output_data) 

In [29]:
df_nmf_topN.to_csv("output_data/topic_modeling_nmf_results.csv", index=False)
df_nmf_topN.to_parquet("output_data/topic_modeling_nmf_results.parquet", index=False)

In [34]:
sentiment_clf(["I need to check my savings account balance."])

[{'label': 'positive', 'score': 0.999242901802063}]